# Извлечение онтологии с применением априиорной структуры

In [64]:
# pip install ontogpt linkml rdflib owlready2 pandas numpy pyyaml

In [2]:
# pip install robotframework-owl

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement robotframework-owl (from versions: none)
ERROR: No matching distribution found for robotframework-owl


## Генерация LinkML схемы на основе GraphRAG данных

In [2]:
import pandas as pd
import yaml
import json
from typing import Dict, List, Set
import re

### Загрузка данных

In [7]:
import os

os.getcwd()

'C:\\Users\\glvv2\\vkr'

In [11]:
entities_df = pd.read_parquet("entities.parquet")

In [12]:
entities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2706 non-null   object 
 1   human_readable_id  2706 non-null   int64  
 2   title              2706 non-null   object 
 3   type               2706 non-null   object 
 4   description        2706 non-null   object 
 5   text_unit_ids      2706 non-null   object 
 6   frequency          2706 non-null   int64  
 7   degree             2706 non-null   int32  
 8   x                  2556 non-null   float64
 9   y                  2556 non-null   float64
dtypes: float64(2), int32(1), int64(2), object(5)
memory usage: 201.0+ KB


In [16]:
entities_df.head()

,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,02a72574-14b0-4f83-b5ba-4eb26c84e880,0,УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ,COMMAND,"Функции, позволяющие управлять визуализацией м...",[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,1,4,0.0,0.0
1,747e7b9f-db56-40da-b0eb-7c431feceec5,1,3D-СЦЕНА,3D SCENE,3D-СЦЕНА представляет собой трехмерное простра...,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,35,34,0.0,0.0
2,f2371ed6-6b5e-4cd2-8d34-7c8a59329b86,2,ПАНЕЛЬ «ВИД»,TOOL,ПАНЕЛЬ «ВИД» представляет собой специализирова...,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,2,11,0.0,0.0
3,fd0fa5f6-b495-4bba-a241-eb07d952e296,3,СТИЛЬ ОТОБРАЖЕНИЯ,PARAMETER,"Параметр, определяющий стиль, в котором объект...",[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,1,1,0.0,0.0
4,b7a6d186-3f63-4124-b350-5ff56a6d14cb,4,МЕТОД ПРОЕКЦИРОВАНИЯ,PARAMETER,"Метод, используемый для проецирования изображе...",[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,1,1,0.0,0.0


In [20]:
entities_df

,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,02a72574-14b0-4f83-b5ba-4eb26c84e880,0,УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ,COMMAND,"Функции, позволяющие управлять визуализацией м...",[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,1,4,0.0,0.0
1,747e7b9f-db56-40da-b0eb-7c431feceec5,1,3D-СЦЕНА,3D SCENE,3D-СЦЕНА представляет собой трехмерное простра...,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,35,34,0.0,0.0
2,f2371ed6-6b5e-4cd2-8d34-7c8a59329b86,2,ПАНЕЛЬ «ВИД»,TOOL,ПАНЕЛЬ «ВИД» представляет собой специализирова...,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,2,11,0.0,0.0
3,fd0fa5f6-b495-4bba-a241-eb07d952e296,3,СТИЛЬ ОТОБРАЖЕНИЯ,PARAMETER,"Параметр, определяющий стиль, в котором объект...",[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,1,1,0.0,0.0
4,b7a6d186-3f63-4124-b350-5ff56a6d14cb,4,МЕТОД ПРОЕКЦИРОВАНИЯ,PARAMETER,"Метод, используемый для проецирования изображе...",[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2701,a68e7120-3692-4f22-bfe0-08f328eb1264,2701,БЛОК «ДОБАВЛЯЕМЫЕ ОБЪЕКТЫ»,PARAMETER,"Блок, предназначенный для выбора объектов, кот...",[9735bcb4cd0c720636948bbe7f1447b365ed6d3121b44...,1,1,0.0,0.0
2702,e74ab064-0279-42c5-a2b9-b4cdd3919f47,2702,БЛОК «МОДИФИЦИРУЕМЫЕ ОБЪЕКТЫ»,PARAMETER,"Блок, предназначенный для выбора объектов, кот...",[9735bcb4cd0c720636948bbe7f1447b365ed6d3121b44...,1,1,0.0,0.0
2703,7e4e433b-b64e-4c60-a14a-1cd9a85aaa63,2703,ФЛАГ «АКТИВИРОВАТЬ СЦЕНУ»,PARAMETER,"Флаг, который позволяет сделать созданную сцен...",[9735bcb4cd0c720636948bbe7f1447b365ed6d3121b44...,1,1,0.0,0.0
2704,d385e00f-cce8-4025-9852-18ce7a4a928d,2704,БЛОК «МОДИФИЦИРУЕМЫЕ ОБЪЕКТЫ«,,,[9735bcb4cd0c720636948bbe7f1447b365ed6d3121b44...,1,1,0.0,0.0


In [13]:
entities_df.type.unique()

array(['COMMAND', '3D SCENE', 'TOOL', 'PARAMETER', 'OBJECT', '',
       'INTERFACE', 'DIALOG', 'BUTTON', 'USER ACTION', 'GEOMETRY',
       'CURVE', 'ASSEMBLY', 'MODEL', 'VARIABLE', 'COMMENT', 'VISIBILITY',
       'RELATIONSHIP', 'ERROR MARKING', 'NAVIGATION', 'SURFACE',
       'TRANSFORMATION', 'TEXTURE', 'DIMENSION', 'POINT', 'MENU', 'KEY',
       'FILE', 'DIRECTORY', 'SECTION', 'IMAGE'], dtype=object)

In [14]:
relationships_df = pd.read_parquet("relationships.parquet")

In [15]:
relationships_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3501 entries, 0 to 3500
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3501 non-null   object 
 1   human_readable_id  3501 non-null   int64  
 2   source             3501 non-null   object 
 3   target             3501 non-null   object 
 4   description        3501 non-null   object 
 5   weight             3501 non-null   float64
 6   combined_degree    3501 non-null   int64  
 7   text_unit_ids      3501 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 218.9+ KB


In [17]:
relationships_df.head()

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,4f65e304-2128-49b4-b2f3-80b0dfeaf2fb,0,УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ,ПАНЕЛЬ «ВИД»,Функции управления визуализацией модели распол...,9.0,15,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...
1,d1543bc8-33d0-43cb-9125-964c37d035ff,1,УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ,СТИЛЬ ОТОБРАЖЕНИЯ,Управление визуализацией модели включает выбор...,7.0,5,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...
2,b46f805b-9dc4-451e-97eb-9602b7afea03,2,УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ,МЕТОД ПРОЕКЦИРОВАНИЯ,Управление визуализацией модели включает выбор...,7.0,5,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...
3,569b48fc-36e1-4eff-be4f-e68cb406e728,3,УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ,МАСШТАБ ИЗОБРАЖЕНИЯ,Управление визуализацией модели включает функц...,1.0,5,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...
4,f84b6ae1-ab3d-4066-84fb-7cc0f1b90c80,4,3D-СЦЕНА,ПАНЕЛЬ «ВИД»,Панель «Вид» управляет визуализацией объектов ...,8.0,45,[cbc5a6c172c48df09239519f9b9f7cff921178a850a50...


In [18]:
community_reports_df = pd.read_parquet("community_reports.parquet")

In [19]:
community_reports_df.head()

,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,a2632f405fcc46408691463ae32c4480,271,271,2,37,[],КРИВАЯ and its Associated Modes and Parameters,The community centers around the entity КРИВАЯ...,# КРИВАЯ and its Associated Modes and Paramete...,9.0,The high impact severity rating reflects the c...,[{'explanation': 'КРИВАЯ is a central entity i...,"{\n ""title"": ""КРИВАЯ and its Associated Mod...",2025-07-25,26
1,2dcce166f98b462abbf7769f247473e8,272,272,2,37,[],КРИВАЯ ПО ФОРМУЛЕ and СК СТРАНИЦЫ,The community focuses on the КРИВАЯ ПО ФОРМУЛЕ...,# КРИВАЯ ПО ФОРМУЛЕ and СК СТРАНИЦЫ\n\nThe com...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'КРИВАЯ ПО ФОРМУЛЕ is a cruci...,"{\n ""title"": ""КРИВАЯ ПО ФОРМУЛЕ and СК СТРА...",2025-07-25,2
2,3747368e912147a59f12cc6641a9490d,273,273,2,37,[],Formula Fields and Curve Creation,The community focuses on the specialized Formu...,# Formula Fields and Curve Creation\n\nThe com...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'Formula Fields are specializ...,"{\n ""title"": ""Formula Fields and Curve Crea...",2025-07-25,2
3,0fb0041b9f2c44cca4f579512c62486a,274,274,2,37,[],Scaling Point and Scaling Method in CAD Docume...,The community focuses on the relationship betw...,# Scaling Point and Scaling Method in CAD Docu...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'The Scaling Point is a cruci...,"{\n ""title"": ""Scaling Point and Scaling Met...",2025-07-25,2
4,fb9cba0665e047e4ba6f086d3cc8b183,275,275,2,37,[],Флаговая Кнопка «По Точкам» и Опция «Непрерывн...,The community focuses on the interaction betwe...,# Флаговая Кнопка «По Точкам» и Опция «Непреры...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'Флаговая Кнопка «По Точкам» ...,"{\n ""title"": ""Флаговая Кнопка «По Точкам» и...",2025-07-25,2


In [26]:
type_entities = entities_df[entities_df['type'] == 'COMMAND']
len(type_entities)

570

In [28]:
type_entities = entities_df[entities_df['type'] == 'PARAMETER']
len(type_entities)

949

In [55]:
relationship_descriptions = relationships_df['description'].value_counts().head(10)
relationship_descriptions

description
Панель является одним из типов инструментальных панелей, используемых в CAD                       5
Способ построения дуги окружности выбирается из выпадающего списка под пиктограммой команды       5
Способ построения окружности выбирается из выпадающего списка под пиктограммой команды            5
Выбор способа построения дуги окружности осуществляется в диалоге параметров команды              5
Обе команды относятся к процессу копирования 2D-объектов в CAD                                    3
После запуска команды открывается диалог параметров для настройки создания поверхности            2
Точка 3 используется для задания направления оси Y в режиме «Система координат по трём точкам»    2
Элементы сборки могут быть добавлены в электронную структуру.                                     2
Команда «Сечение поверхностью» вызывается из выпадающего списка на панели «Лента»                 2
Кнопка в диалоге параметров предназначена для отображения списка свободных букв         

### Построение LinkML схемы

In [124]:
class SimpleLinkMLGenerator:
    """
    Класс построения LinkML схемы для OntoGPT
    """
    
    def __init__(self, entities_path: str, relationships_path: str, max_entity_types: int = 10, max_relations: int = 10):
        self.entities_df = pd.read_parquet(entities_path)
        self.relationships_df = pd.read_parquet(relationships_path)
        self.max_entity_types = max_entity_types
        self.max_relations = max_relations
        
    def generate_simple_schema(self) -> str:
        """Вызов методов построения"""
        
        print("Анализ типов сущностей...")
        entity_types = self._analyze_entity_types()
        
        print("Анализ отношений...")
        relationship_types = self._analyze_relationship_types()
        
        print("Построение схемы...")
        schema = self._build_linkml_schema(entity_types, relationship_types)
        
        return yaml.dump(schema, sort_keys=False, allow_unicode=True, indent=2, default_flow_style=False)
    
    def _analyze_entity_types(self) -> Dict:
        """Анализ топ N типов сущностей"""
        
        entity_types = {}
        
        # Берем только топ N наиболее частых типов
        type_counts = self.entities_df['type'].value_counts().head(self.max_entity_types)
        
        for entity_type, count in type_counts.items():
            if pd.isna(entity_type) or entity_type == '':
                continue
                
            type_entities = self.entities_df[self.entities_df['type'] == entity_type]
            
            entity_types[entity_type] = {
                "description": f"Сущности типа {entity_type} из CAD системы",
                "count": count,
                "sample_titles": type_entities['title'].head(3).tolist()
            }
        
        return entity_types
    
    def _analyze_relationship_types(self) -> Dict[str, Dict]:
        """Анализ топ N типов отношений"""
        
        relationship_types = {}
        
        # Берем только топ N наиболее частых отношений
        relationship_descriptions = self.relationships_df['description'].value_counts().head(self.max_relations)
        
        for desc, count in relationship_descriptions.items():
            if pd.isna(desc) or desc == '':
                continue
                
            rel_name = self._create_relationship_name(desc)
            
            # Находим типы источника и цели
            example_rels = self.relationships_df[
                self.relationships_df['description'] == desc
            ].head(5)
            
            source_types = set()
            target_types = set()
            
            for _, rel in example_rels.iterrows():
                source_entity = self.entities_df[self.entities_df['id'] == rel['source']]
                target_entity = self.entities_df[self.entities_df['id'] == rel['target']]
                
                if not source_entity.empty:
                    source_types.add(source_entity.iloc[0]['type'])
                if not target_entity.empty:
                    target_types.add(target_entity.iloc[0]['type'])
            
            relationship_types[rel_name] = {
                "description": desc,
                "count": count,
                "source_types": list(source_types),
                "target_types": list(target_types)
            }
        
        return relationship_types

    def _create_relationship_name(self, description: str) -> str:
        """Создание простого имени отношения"""
        
        if not description or pd.isna(description):
            return "related_to"
        
        # Упрощенная версия - берем первые 2-3 значимых слова
        clean = re.sub(r'[^а-яА-Яa-zA-Z0-9\s]', ' ', str(description).lower())
        words = clean.split()[:3]
        
        transliterated = []
        for word in words:
            trans = self._transliterate(word)
            if trans and len(trans) > 2:
                transliterated.append(trans)
        
        result = '_'.join(transliterated[:2]) if transliterated else "related_to"
        return result if result else "related_to"
    
    def _build_linkml_schema(self, entity_types: Dict, relationship_types: Dict) -> Dict:
        """Построение LinkML схемы по примеру OntoGPT"""
        
        schema = {
            "id": "https://example.org/cad-graphrag-ontology",
            "name": "CADGraphRAGOntology",
            "title": "CAD GraphRAG Ontology",
            "description": "Автоматически сгенерированная онтология из GraphRAG данных CAD системы",
            "license": "https://creativecommons.org/publicdomain/zero/1.0/",
            "imports": [
                "linkml:types",
                "core"
            ],
            "prefixes": {
                "linkml": "https://w3id.org/linkml/",
                "cad": "https://example.org/cad/",
                "schema": "http://schema.org/"
            },
            "default_prefix": "cad",
            "default_range": "string",
            "classes": {},
            "slots": {}
        }
        
        # 1. КОРНЕВОЙ КЛАСС с атрибутами для списков сущностей
        root_attributes = {}
        
        # Атрибуты для сущностей
        for entity_type in entity_types.keys():
            class_name = self._type_to_class_name(entity_type)
            attr_name = f"{class_name.lower()}_list"
            
            root_attributes[attr_name] = {
                "description": f"semicolon-separated list of {entity_type} entities",
                "multivalued": True,
                "range": class_name
            }
        
        # Атрибуты для отношений  
        for rel_name in relationship_types.keys():
            rel_class_name = self._to_class_name(rel_name)
            attr_name = f"{rel_name}_list"
            
            root_attributes[attr_name] = {
                "description": f"semicolon-separated list of {rel_name} relationships",
                "multivalued": True,
                "range": rel_class_name
            }
        
        schema["classes"]["CADExtractionResult"] = {
            "tree_root": True,
            "description": "Result of extracting CAD entities and relationships from documentation",
            "attributes": root_attributes
        }
        
        # 2. КЛАССЫ СУЩНОСТЕЙ - НЕ переопределяем атрибуты NamedEntity!
        for entity_type, type_info in entity_types.items():
            class_name = self._type_to_class_name(entity_type)
            
            # Простое определение класса без переопределения атрибутов
            schema["classes"][class_name] = {
                "is_a": "NamedEntity",
                "description": type_info["description"],
                "id_prefixes": ["CAD"],
                "annotations": {
                    "annotators": "sqlite:obo:mesh"
                }
            }
        
        # 3. КЛАССЫ ОТНОШЕНИЙ
        for rel_name, rel_info in relationship_types.items():
            rel_class_name = self._to_class_name(rel_name)
            
            # Определяем типы источника и цели
            source_type = rel_info["source_types"][0] if rel_info["source_types"] else None
            target_type = rel_info["target_types"][0] if rel_info["target_types"] else None
            
            source_class = self._type_to_class_name(source_type) if source_type else "NamedEntity"
            target_class = self._type_to_class_name(target_type) if target_type else "NamedEntity"
            
            schema["classes"][rel_class_name] = {
                "is_a": "CompoundExpression",
                "description": f"Relationship: {rel_info['description']}",
                "attributes": {
                    "source_entity": {
                        "description": "The source entity in the relationship",
                        "range": source_class,
                        "annotations": {
                            "prompt": "the first entity in the relationship"
                        }
                    },
                    "target_entity": {
                        "description": "The target entity in the relationship",
                        "range": target_class,
                        "annotations": {
                            "prompt": "the second entity in the relationship"
                        }
                    }
                }
            }
        
        return schema
    
    def _type_to_class_name(self, entity_type: str) -> str:
        """Преобразование в CamelCase имя класса"""
        
        if not entity_type or pd.isna(entity_type):
            return "UnknownEntity"
        
        transliterated = self._transliterate(str(entity_type))
        clean = re.sub(r'[^a-zA-Z0-9]', '_', transliterated)
        clean = re.sub(r'_+', '_', clean).strip('_')
        
        if not clean:
            return "UnknownEntity"
        
        if clean[0].isdigit():
            clean = "Entity_" + clean
        
        words = clean.split('_')
        return ''.join(word.capitalize() for word in words if word)
    
    def _to_class_name(self, name: str) -> str:
        """Преобразование имени отношения в класс"""
        
        if not name or pd.isna(name):
            return "RelatedToRelationship"
        
        words = str(name).split('_')
        return ''.join(word.capitalize() for word in words if word) + "Relationship"

    def _transliterate(self, text: str) -> str:
        """Транслитерация кириллицы"""
        
        if not text or pd.isna(text):
            return ""
        
        text = str(text).strip()
        
        translit_map = {
            'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'e',
            'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
            'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
            'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch',
            'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e', 'ю': 'yu', 'я': 'ya',
            'А': 'A', 'Б': 'B', 'В': 'V', 'Г': 'G', 'Д': 'D', 'Е': 'E', 'Ё': 'E',
            'Ж': 'Zh', 'З': 'Z', 'И': 'I', 'Й': 'Y', 'К': 'K', 'Л': 'L', 'М': 'M',
            'Н': 'N', 'О': 'O', 'П': 'P', 'Р': 'R', 'С': 'S', 'Т': 'T', 'У': 'U',
            'Ф': 'F', 'Х': 'Kh', 'Ц': 'Ts', 'Ч': 'Ch', 'Ш': 'Sh', 'Щ': 'Shch',
            'Ъ': '', 'Ы': 'Y', 'Ь': '', 'Э': 'E', 'Ю': 'Yu', 'Я': 'Ya'
        }
        
        result = []
        for char in text:
            if char in translit_map:
                trans = translit_map[char]
                if trans:
                    result.append(trans)
            elif char.isalnum():
                result.append(char)
            else:
                result.append('_')
        
        result_str = ''.join(result)
        result_str = re.sub(r'_+', '_', result_str).strip('_')
        
        return result_str.lower()


In [104]:
# class SimpleLinkMLGenerator:
#     """
#     Класс построения простой LinkML схемы на основе типов сущностей и отношений
#     """
    
#     def __init__(self, entities_path: str, relationships_path: str):
#         self.entities_df = pd.read_parquet(entities_path)
#         self.relationships_df = pd.read_parquet(relationships_path)
        
#     def generate_simple_schema(self) -> str:
#         """Вызов методов построения"""
        
#         print("Анализ типов сущностей...")
#         entity_types = self._analyze_entity_types()
        
#         print("Анализ отношений...")
#         relationship_types = self._analyze_relationship_types()
        
#         print("Построение схемы...")
#         schema = self._build_linkml_schema(entity_types, relationship_types)
        
#         return yaml.dump(schema, sort_keys=False, allow_unicode=True, indent=2)
    
#     def _analyze_entity_types(self) -> Dict[str, Dict]:
#         """Анализ типов сущностей и их атрибутов"""
        
#         entity_types = {}
        
#         # Группируем сущности по типу
#         for entity_type in self.entities_df['type'].unique():
#             if pd.isna(entity_type) or entity_type == '':
#                 continue
                
#             type_entities = self.entities_df[self.entities_df['type'] == entity_type]
            
#             # Анализируем атрибуты для этого типа
#             attributes = self._extract_attributes_for_type(type_entities, entity_type)
            
#             entity_types[entity_type] = {
#                 "description": f"Сущности типа {entity_type} из CAD системы",
#                 "attributes": attributes,
#                 "count": len(type_entities),
#                 "sample_titles": type_entities['title'].head(3).tolist()
#             }
        
#         return entity_types
    
#     def _extract_attributes_for_type(self, type_entities: pd.DataFrame, entity_type: str) -> Dict[str, Dict]:
#         """Извлечение атрибутов для конкретного типа сущностей"""
        
#         attributes = {
#             "id": {
#                 "range": "string",
#                 "description": "Уникальный идентификатор сущности",
#                 "required": True
#             },
#             "title": {
#                 "range": "string", 
#                 "description": "Название сущности",
#                 "required": True
#             },
#             "description": {
#                 "range": "string",
#                 "description": "Описание сущности", 
#                 "required": False
#             }
#         }
        
#         # Добавляем дополнительные атрибуты на основе анализа
#         if 'frequency' in type_entities.columns:
#             attributes["frequency"] = {
#                 "range": "integer",
#                 "description": "Частота встречаемости в документации",
#                 "required": False
#             }
        
#         if 'degree' in type_entities.columns:
#             attributes["degree"] = {
#                 "range": "integer", 
#                 "description": "Степень связности в графе",
#                 "required": False
#             }
        
#         # Анализ описаний для выявления специфических атрибутов
#         specific_attrs = self._extract_specific_attributes(type_entities, entity_type)
#         attributes.update(specific_attrs)
        
#         return attributes
    
#     def _extract_specific_attributes(self, type_entities: pd.DataFrame, entity_type: str) -> Dict[str, Dict]:
#         """Извлечение специфических атрибутов из описаний сущностей"""
        
#         specific_attrs = {}
        
#         # Анализируем описания для выявления паттернов
#         descriptions = type_entities['description'].dropna()
        
#         if len(descriptions) == 0:
#             return specific_attrs
        
#         # Эвристики для разных типов сущностей
#         if entity_type == 'PARAMETER':
#             # Для параметров ищем числовые значения и единицы измерения
#             specific_attrs["value"] = {
#                 "range": "string",
#                 "description": "Значение параметра",
#                 "required": False
#             }
#             # specific_attrs["unit"] = {
#             #     "range": "string",
#             #     "description": "Единица измерения параметра",
#             #     "required": False
#             # }
        
#         elif entity_type == 'COMMAND':
#             # Для команд ищем связанные элементы интерфейса
#             specific_attrs["ui_location"] = {
#                 "range": "string",
#                 "description": "Расположение в интерфейсе",
#                 "required": False
#             }
        
#         elif entity_type in ['3D SCENE', 'GEOMETRY', 'OBJECT']:
#             # Для геометрических объектов
#             specific_attrs["geometry_type"] = {
#                 "range": "string",
#                 "description": "Тип геометрического объекта",
#                 "required": False
#             }
        
#         return specific_attrs
    
#     def _analyze_relationship_types(self) -> Dict[str, Dict]:
#         """Анализ типов отношений между сущностями"""
        
#         relationship_types = {}
        
#         # Анализируем описания отношений для группировки
#         relationship_descriptions = self.relationships_df['description'].value_counts().head(10)
        
#         for desc, count in relationship_descriptions.items():
#             # Создаем имя для типа отношения на основе описания
#             rel_name = self._create_relationship_name(desc)
            
#             # Находим примеры source и target типов для этого отношения
#             example_rels = self.relationships_df[
#                 self.relationships_df['description'] == desc
#             ].head(5)
            
#             source_types = set()
#             target_types = set()
            
#             for _, rel in example_rels.iterrows():
#                 source_entity = self.entities_df[self.entities_df['id'] == rel['source']]
#                 target_entity = self.entities_df[self.entities_df['id'] == rel['target']]
                
#                 if not source_entity.empty:
#                     source_types.add(source_entity.iloc[0]['type'])
#                 if not target_entity.empty:
#                     target_types.add(target_entity.iloc[0]['type'])
            
#             relationship_types[rel_name] = {
#                 "description": desc,
#                 "count": count,
#                 "source_types": list(source_types),
#                 "target_types": list(target_types)
#             }
        
#         return relationship_types

#     def _create_relationship_name(self, description: str) -> str:
#         """Создание имени отношения из описания с транслитерацией кириллицы"""
        
#         # Транслитерируем описание
#         transliterated_desc = self._transliterate(description)
        
#         # Очистка и нормализация
#         clean_desc = re.sub(r'[^a-zA-Z0-9\s]', '', transliterated_desc.lower())
#         words = clean_desc.split()
        
#         if len(words) == 0:
#             return "related_to"
        
#         # Берем первые 2 значимых слова
#         stop_words = {'i', 'ili', 'a', 'v', 'iz', 'dlya', 'k'}
#         meaningful_words = [w for w in words if w not in stop_words][:2]
        
#         if not meaningful_words:
#             meaningful_words = words[:2]
        
#         return '_'.join(meaningful_words)

#     def _create_relationship_name(self, description: str, max_words: int = 3, max_length: int = 40) -> str:
#         """Создание короткого имени отношения из описания с транслитерацией кириллицы"""
        
#         if not description or pd.isna(description):
#             return "related_to"
        
#         # Удаляем скобки, двоеточия, лишние пробелы
#         clean_desc = re.sub(r'[()\[\]{}<>:;,.!?]', ' ', str(description))
#         clean_desc = re.sub(r'\s+', ' ', clean_desc).strip().lower()
        
#         words = clean_desc.split()
        
#         if len(words) == 0:
#             return "related_to"
        
#         stop_words = {
#             # Русские предлоги и союзы
#             'и', 'или', 'а', 'в', 'во', 'из', 'от', 'до', 'для', 'к', 'на', 'по', 'с', 'со',
#             'у', 'над', 'под', 'перед', 'через', 'без', 'о', 'об', 'около', 'возле'
#         }
        
#         meaningful_words = [word for word in words 
#                            if word not in stop_words and len(word) > 2 and not word.isdigit()]
        
#         # Если нет значимых слов - берем первые слова из оригинального описания
#         if not meaningful_words:
#             meaningful_words = words[:max_words]
        
#         selected_words = []
        
#         # Берем 1-2 слова из начала
#         start_words = [w for w in meaningful_words[:2] if len(w) > 3]
#         selected_words.extend(start_words[:min(2, len(start_words))])
        
#         if len(selected_words) < max_words and len(meaningful_words) > 3:
#             end_words = [w for w in meaningful_words[-2:] if len(w) > 3 and w not in selected_words]
#             selected_words.extend(end_words[:min(max_words - len(selected_words), len(end_words))])
        
#         # Если все еще нет слов - берем любые из описания
#         if not selected_words:
#             selected_words = words[:max_words]
        
#         selected_words = selected_words[:max_words]
        
#         transliterated_words = []
#         for word in selected_words:
#             translit_word = self._transliterate(word.strip())
#             # Удаляем лишние подчеркивания внутри слова
#             translit_word = re.sub(r'_+', '_', translit_word).strip('_')
#             if translit_word:
#                 transliterated_words.append(translit_word)
        
#         if not transliterated_words:
#             return "related_to"
        
#         result_name = '_'.join(transliterated_words[:max_words])
        
#         if len(result_name) > max_length:
#             result_name = result_name[:max_length].rsplit('_', 1)[0]
        
#         result_name = re.sub(r'_+', '_', result_name).strip('_')
        
#         if result_name and result_name[0].isdigit():
#             result_name = f"rel_{result_name}"
        
#         if len(result_name) < 3 or re.match(r'^\d+$', result_name):
#             return "related_to"
        
#         return result_name.lower()
    
#     def _build_linkml_schema(self, entity_types: Dict, relationship_types: Dict) -> Dict:
#         """Построение полной LinkML схемы с корректными URI"""
        
#         schema = {
#             "id": "https://example.org/cad-graphrag-ontology",
#             "name": "CADGraphRAGOntology",
#             "description": "Автоматически сгенерированная онтология из GraphRAG данных CAD системы",
#             "imports": ["linkml:types"],
#             "prefixes": {
#                 "linkml": "https://w3id.org/linkml/",
#                 "cad": "https://example.org/cad/",
#                 "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
#                 "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
#                 "owl": "http://www.w3.org/2002/07/owl#",
#                 "schema": "http://schema.org/"  # Добавляем для возможности реюза
#             },
#             "default_prefix": "cad",
#             "default_range": "string",
#             "classes": {},
#             "slots": {},
#             "enums": {}
#         }
        
#         # Добавляем классы для каждого типа сущности
#         for entity_type, type_info in entity_types.items():
#             class_name = self._type_to_class_name(entity_type)
            
#             # Определяем слоты для этого класса
#             slots = list(type_info["attributes"].keys())
            
#             # Добавляем отношения как слоты (только латинские имена)
#             for rel_name, rel_info in relationship_types.items():
#                 latin_rel_name = self._transliterate(rel_name) if any(ord(c) > 127 for c in rel_name) else rel_name
#                 if entity_type in rel_info["source_types"]:
#                     slots.append(latin_rel_name)
            
#             schema["classes"][class_name] = {
#                 "description": type_info["description"],
#                 "slots": slots,
#                 "class_uri": f"cad:{class_name}",
#                 "notes": f"На основе {type_info['count']} сущностей. Примеры: {', '.join(type_info['sample_titles'])}"
#             }
        
#         # Добавляем слоты для атрибутов
#         all_attributes = {}
#         for type_info in entity_types.values():
#             all_attributes.update(type_info["attributes"])
        
#         for attr_name, attr_info in all_attributes.items():
#             # Для стандартных атрибутов можно использовать существующие ontology terms
#             slot_uri = f"cad:{attr_name}"
#             if attr_name in ['id', 'title', 'description']:
#                 # Реюз стандартных терминов из schema.org где возможно
#                 slot_uri = f"schema:{attr_name}" if attr_name in ['name', 'description'] else f"cad:{attr_name}"
            
#             schema["slots"][attr_name] = {
#                 "description": attr_info["description"],
#                 "range": attr_info["range"],
#                 "slot_uri": slot_uri,
#                 "required": attr_info.get("required", False)
#             }

#         # Добавляем слоты для отношений
#         for rel_name, rel_info in relationship_types.items():
#             # Транслитерируем имя отношения если нужно
#             latin_rel_name = self._transliterate(rel_name) if any(ord(c) > 127 for c in rel_name) else rel_name
            
#             # Создаем короткое имя для URI на основе описания
#             short_rel_name = self._create_relationship_name(rel_info["description"], max_words=3, max_length=30)
            
#             schema["slots"][latin_rel_name] = {
#                 "description": rel_info["description"],
#                 "range": "uriorcurie",
#                 "multivalued": True,
#                 "slot_uri": f"cad:{short_rel_name}",
#                 "comments": f"Частота: {rel_info['count']}. Источники: {rel_info['source_types']}. Цели: {rel_info['target_types']}"
#             }
        
#         return schema
    
#     def _type_to_class_name(self, entity_type: str) -> str:
#         """Преобразование типа сущности в имя класса с транслитерацией кириллицы"""
        
#         # Транслитерируем тип сущности
#         transliterated_type = self._transliterate(entity_type)
        
#         # Замена пробелов и специальных символов
#         clean_type = re.sub(r'[^a-zA-Z0-9]', '_', transliterated_type)
        
#         # Удаляем повторяющиеся подчеркивания
#         clean_type = re.sub(r'_+', '_', clean_type).strip('_')
        
#         # Если тип начинается с цифры, добавляем префикс
#         if clean_type and clean_type[0].isdigit():
#             clean_type = "Class_" + clean_type
        
#         # Если после обработки строка пустая, используем fallback
#         if not clean_type:
#             clean_type = "UnknownEntity"
        
#         return clean_type

#     def _is_valid_curie(self, curie: str) -> bool:
#         """Проверка валидности CURIE"""
#         # Простая проверка основывается на стандарте: prefix:reference
#         pattern = r'^[a-zA-Z_][a-zA-Z0-9_\-]*:[a-zA-Z0-9_\-\.]+$'
#         return re.match(pattern, curie) is not None

#     def _transliterate(self, text: str) -> str:
#         """Транслитерация кириллических символов в латиницу"""

#         if not text or pd.isna(text):
#             return ""
        
#         text = str(text).strip()
        
#         translit_map = {
#             'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'e',
#             'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
#             'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
#             'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch',
#             'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e', 'ю': 'yu', 'я': 'ya',
#             'А': 'A', 'Б': 'B', 'В': 'V', 'Г': 'G', 'Д': 'D', 'Е': 'E', 'Ё': 'E',
#             'Ж': 'Zh', 'З': 'Z', 'И': 'I', 'Й': 'Y', 'К': 'K', 'Л': 'L', 'М': 'M',
#             'Н': 'N', 'О': 'O', 'П': 'P', 'Р': 'R', 'С': 'S', 'Т': 'T', 'У': 'U',
#             'Ф': 'F', 'Х': 'Kh', 'Ц': 'Ts', 'Ч': 'Ch', 'Ш': 'Sh', 'Щ': 'Shch',
#             'Ъ': '', 'Ы': 'Y', 'Ь': '', 'Э': 'E', 'Ю': 'Yu', 'Я': 'Ya'
#         }
        
#         result_chars = []
#         prev_was_space = False
        
#         for char in text:
#             if char in translit_map:
#                 translit_char = translit_map[char]
#                 if translit_char:  # если символ не был удалён (как ъ, ь)
#                     result_chars.append(translit_char)
#                     prev_was_space = False
#             elif char.isalnum():
#                 result_chars.append(char)
#                 prev_was_space = False
#             elif char.isspace() or char in ['-', '_', '/']:
#                 # Заменяем разделители на одно подчеркивание
#                 if not prev_was_space:
#                     result_chars.append('_')
#                     prev_was_space = True
#             # Игнорируем остальные символы
        
#         result = ''.join(result_chars)
        
#         # Удаляем повторяющиеся подчеркивания и лишние в начале/конце
#         result = re.sub(r'_+', '_', result).strip('_')
        
#         # Разбиваем на слова и оставляем только те, которые содержат буквы
#         words = result.split('_')
#         valid_words = [word for word in words if re.search(r'[a-zA-Z]', word)]
        
#         if not valid_words:
#             return result
        
#         return '_'.join(valid_words).lower()
        

In [100]:
# class SimpleLinkMLGenerator:
#     """
#     Класс построения простой LinkML схемы на основе типов сущностей и отношений
#     с поддержкой валидных CURIE идентификаторов
#     """
    
#     def __init__(self, entities_path: str, relationships_path: str):
#         self.entities_df = pd.read_parquet(entities_path)
#         self.relationships_df = pd.read_parquet(relationships_path)
        
#     def generate_simple_schema(self) -> str:
#         """Вызов методов построения"""
        
#         print("Анализ типов сущностей...")
#         entity_types = self._analyze_entity_types()
        
#         print("Анализ отношений...")
#         relationship_types = self._analyze_relationship_types()
        
#         print("Построение схемы...")
#         schema = self._build_linkml_schema(entity_types, relationship_types)
        
#         return yaml.dump(schema, sort_keys=False, allow_unicode=True, indent=2)
    
#     def _analyze_entity_types(self) -> Dict[str, Dict]:
#         """Анализ типов сущностей и их атрибутов"""
        
#         entity_types = {}
        
#         # Группируем сущности по типу
#         for entity_type in self.entities_df['type'].unique():
#             if pd.isna(entity_type) or entity_type == '':
#                 continue
                
#             type_entities = self.entities_df[self.entities_df['type'] == entity_type]
            
#             # Анализируем атрибуты для этого типа
#             attributes = self._extract_attributes_for_type(type_entities, entity_type)
            
#             entity_types[entity_type] = {
#                 "description": f"Сущности типа {entity_type} из CAD системы",
#                 "attributes": attributes,
#                 "count": len(type_entities),
#                 "sample_titles": type_entities['title'].head(3).tolist()
#             }
        
#         return entity_types
    
#     def _extract_attributes_for_type(self, type_entities: pd.DataFrame, entity_type: str) -> Dict[str, Dict]:
#         """Извлечение атрибутов для конкретного типа сущностей"""
        
#         attributes = {
#             "id": {
#                 "range": "string",
#                 "description": "Уникальный идентификатор сущности",
#                 "required": True
#             },
#             "title": {
#                 "range": "string", 
#                 "description": "Название сущности",
#                 "required": True
#             },
#             "description": {
#                 "range": "string",
#                 "description": "Описание сущности", 
#                 "required": False
#             }
#         }
        
#         # Добавляем дополнительные атрибуты на основе анализа
#         if 'frequency' in type_entities.columns:
#             attributes["frequency"] = {
#                 "range": "integer",
#                 "description": "Частота встречаемости в документации",
#                 "required": False
#             }
        
#         if 'degree' in type_entities.columns:
#             attributes["degree"] = {
#                 "range": "integer", 
#                 "description": "Степень связности в графе",
#                 "required": False
#             }
        
#         # Анализ описаний для выявления специфических атрибутов
#         specific_attrs = self._extract_specific_attributes(type_entities, entity_type)
#         attributes.update(specific_attrs)
        
#         return attributes
    
#     def _extract_specific_attributes(self, type_entities: pd.DataFrame, entity_type: str) -> Dict[str, Dict]:
#         """Извлечение специфических атрибутов из описаний сущностей"""
        
#         specific_attrs = {}
        
#         # Анализируем описания для выявления паттернов
#         descriptions = type_entities['description'].dropna()
        
#         if len(descriptions) == 0:
#             return specific_attrs
        
#         # Эвристики для разных типов сущностей
#         if entity_type == 'PARAMETER':
#             # Для параметров ищем числовые значения и единицы измерения
#             specific_attrs["value"] = {
#                 "range": "string",
#                 "description": "Значение параметра",
#                 "required": False
#             }
        
#         elif entity_type == 'COMMAND':
#             # Для команд ищем связанные элементы интерфейса
#             specific_attrs["ui_location"] = {
#                 "range": "string",
#                 "description": "Расположение в интерфейсе",
#                 "required": False
#             }
        
#         elif entity_type in ['3D SCENE', 'GEOMETRY', 'OBJECT']:
#             # Для геометрических объектов
#             specific_attrs["geometry_type"] = {
#                 "range": "string",
#                 "description": "Тип геометрического объекта",
#                 "required": False
#             }
        
#         return specific_attrs
    
#     def _analyze_relationship_types(self) -> Dict[str, Dict]:
#         """Анализ типов отношений между сущностями"""
        
#         relationship_types = {}
        
#         # Анализируем описания отношений для группировки
#         relationship_descriptions = self.relationships_df['description'].value_counts().head(10)
        
#         for desc, count in relationship_descriptions.items():
#             # Создаем имя для типа отношения на основе описания
#             rel_name = self._create_relationship_name(desc)
            
#             # Находим примеры source и target типов для этого отношения
#             example_rels = self.relationships_df[
#                 self.relationships_df['description'] == desc
#             ].head(5)
            
#             source_types = set()
#             target_types = set()
            
#             for _, rel in example_rels.iterrows():
#                 source_entity = self.entities_df[self.entities_df['id'] == rel['source']]
#                 target_entity = self.entities_df[self.entities_df['id'] == rel['target']]
                
#                 if not source_entity.empty:
#                     source_types.add(source_entity.iloc[0]['type'])
#                 if not target_entity.empty:
#                     target_types.add(target_entity.iloc[0]['type'])
            
#             relationship_types[rel_name] = {
#                 "description": desc,
#                 "count": count,
#                 "source_types": list(source_types),
#                 "target_types": list(target_types)
#             }
        
#         return relationship_types
    
#     def _create_relationship_name(self, description: str) -> str:
#         """Создание имени отношения из описания с использованием только латиницы"""
        
#         # Транслитерация кириллицы в латиницу
#         translit_map = {
#             'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'e',
#             'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
#             'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
#             'ф': 'f', 'х': 'h', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'sch',
#             'ы': 'y', 'э': 'e', 'ю': 'yu', 'я': 'ya',
#             ' ': '_', '-': '_'
#         }
        
#         # Приводим к нижнему регистру и транслитерируем
#         clean_desc = description.lower()
#         clean_desc = ''.join(translit_map.get(c, c) for c in clean_desc)
        
#         # Удаляем все не-ASCII символы и оставляем только буквы, цифры и подчеркивания
#         clean_desc = re.sub(r'[^a-zA-Z0-9_]', '', clean_desc)
        
#         words = clean_desc.split('_')
        
#         if len(words) == 0:
#             return "related_to"
        
#         # Берем первые 2-3 значимых слова
#         stop_words = {'and', 'or', 'a', 'in', 'of', 'for', 'to', 'the', 'is', 'are'}
#         meaningful_words = [w for w in words if w not in stop_words and len(w) > 1][:2]
        
#         if not meaningful_words:
#             meaningful_words = words[:2]
        
#         result = '_'.join(meaningful_words)
        
#         # Если результат пустой, используем запасной вариант
#         if not result:
#             result = "related_to"
            
#         return result
    
#     def _normalize_identifier(self, identifier: str) -> str:
#         """Нормализация идентификатора для использования в CURIE"""
        
#         # Транслитерация кириллицы в латиницу
#         translit_map = {
#             'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'e',
#             'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
#             'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
#             'ф': 'f', 'х': 'h', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'sch',
#             'ы': 'y', 'э': 'e', 'ю': 'yu', 'я': 'ya',
#             ' ': '_', '-': '_'
#         }
        
#         # Приводим к нижнему регистру и транслитерируем
#         normalized = identifier.lower()
#         normalized = ''.join(translit_map.get(c, c) for c in normalized)
        
#         # Удаляем все недопустимые символы
#         normalized = re.sub(r'[^a-zA-Z0-9_]', '', normalized)
        
#         # Если идентификатор начинается с цифры, добавляем префикс
#         if normalized and normalized[0].isdigit():
#             normalized = "id_" + normalized
            
#         # Если идентификатор пустой после нормализации, создаем запасной
#         if not normalized:
#             normalized = "unknown_" + str(hash(identifier))[:8]
            
#         return normalized
    
#     def _is_valid_curie(self, curie: str) -> bool:
#         """Проверка валидности CURIE"""
#         pattern = r'^[a-zA-Z_][a-zA-Z0-9_\-]*:[a-zA-Z0-9_\-\.]+$'
#         return re.match(pattern, curie) is not None
    
#     def _build_linkml_schema(self, entity_types: Dict, relationship_types: Dict) -> Dict:
#         """Построение полной LinkML схемы с валидными CURIE"""
        
#         schema = {
#             "id": "https://example.org/cad-graphrag-ontology",
#             "name": "CADGraphRAGOntology",
#             "description": "Автоматически сгенерированная онтология из GraphRAG данных CAD системы",
#             "imports": ["linkml:types"],
#             "prefixes": {
#                 "linkml": "https://w3id.org/linkml/",
#                 "cad": "https://example.org/cad/",
#                 "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
#                 "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
#                 "owl": "http://www.w3.org/2002/07/owl#",
#                 "schema": "http://schema.org/",
#                 "skos": "http://www.w3.org/2004/02/skos/core#"
#             },
#             "default_prefix": "cad",
#             "default_range": "string",
#             "classes": {},
#             "slots": {},
#             "enums": {}
#         }
        
#         # Добавляем классы для каждого типа сущности
#         for entity_type, type_info in entity_types.items():
#             class_name = self._type_to_class_name(entity_type)
            
#             # Определяем слоты для этого класса
#             slots = list(type_info["attributes"].keys())
            
#             # Добавляем отношения как слоты (используем нормализованные имена)
#             for rel_name, rel_info in relationship_types.items():
#                 normalized_rel_name = self._normalize_identifier(rel_name)
#                 if entity_type in rel_info["source_types"]:
#                     slots.append(normalized_rel_name)
            
#             # Используем стандартные URI из schema.org где это уместно
#             class_uri = self._get_class_uri(entity_type)
            
#             schema["classes"][class_name] = {
#                 "description": type_info["description"],
#                 "slots": slots,
#                 "class_uri": class_uri,
#                 "notes": f"На основе {type_info['count']} сущностей. Примеры: {', '.join(type_info['sample_titles'])}"
#             }
        
#         # Добавляем слоты для атрибутов
#         all_attributes = {}
#         for type_info in entity_types.values():
#             all_attributes.update(type_info["attributes"])
        
#         for attr_name, attr_info in all_attributes.items():
#             # Нормализуем имя атрибута для CURIE
#             normalized_attr_name = self._normalize_identifier(attr_name)
#             slot_uri = self._get_slot_uri(attr_name, normalized_attr_name)
            
#             schema["slots"][normalized_attr_name] = {
#                 "description": attr_info["description"],
#                 "range": attr_info["range"],
#                 "slot_uri": slot_uri,
#                 "required": attr_info.get("required", False)
#             }
        
#         # Добавляем слоты для отношений
#         for rel_name, rel_info in relationship_types.items():
#             normalized_rel_name = self._normalize_identifier(rel_name)
#             slot_uri = f"cad:{normalized_rel_name}"
            
#             # Проверяем валидность CURIE
#             if not self._is_valid_curie(slot_uri):
#                 # Если невалидно, используем запасной вариант
#                 slot_uri = f"cad:rel_{hash(rel_name) % 10000:04d}"
            
#             schema["slots"][normalized_rel_name] = {
#                 "description": rel_info["description"],
#                 "range": "uriorcurie",
#                 "multivalued": True,
#                 "slot_uri": slot_uri,
#                 "comments": f"Частота: {rel_info['count']}. Источники: {rel_info['source_types']}. Цели: {rel_info['target_types']}"
#             }
        
#         return schema
    
#     def _get_class_uri(self, entity_type: str) -> str:
#         """Получение URI класса с использованием стандартных онтологий где это возможно"""
        
#         class_mappings = {
#             'PARAMETER': 'schema:Property',
#             'COMMAND': 'schema:Action',
#             'OBJECT': 'schema:Thing',
#             'TOOL': 'schema:Tool',
#             'INTERFACE': 'schema:UserInterface',
#             'DIALOG': 'schema:UserInterface',
#             'BUTTON': 'schema:UserInterface',
#             'USER_ACTION': 'schema:Action',
#             'MODEL': 'schema:CreativeWork',
#             'FILE': 'schema:MediaObject',
#             'IMAGE': 'schema:ImageObject',
#             'MENU': 'schema:UserInterface'
#         }
        
#         class_name = self._type_to_class_name(entity_type)
        
#         # Если есть маппинг на стандартную онтологию, используем его
#         if entity_type in class_mappings:
#             return class_mappings[entity_type]
#         else:
#             # Иначе используем наш префикс с нормализованным именем
#             return f"cad:{class_name}"
    
#     def _get_slot_uri(self, attr_name: str, normalized_name: str) -> str:
#         """Получение URI слота с использованием стандартных онтологий где это возможно"""
        
#         slot_mappings = {
#             'id': 'schema:identifier',
#             'title': 'schema:name',
#             'description': 'schema:description',
#             'value': 'schema:value',
#             'frequency': 'schema:usageCount'
#         }
        
#         # Если есть маппинг на стандартную онтологию, используем его
#         if attr_name in slot_mappings:
#             return slot_mappings[attr_name]
#         else:
#             # Иначе используем наш префикс с нормализованным именем
#             slot_uri = f"cad:{normalized_name}"
#             # Проверяем валидность
#             if not self._is_valid_curie(slot_uri):
#                 slot_uri = f"cad:attr_{hash(attr_name) % 10000:04d}"
#             return slot_uri
    
#     def _type_to_class_name(self, entity_type: str) -> str:
#         """Преобразование типа сущности в имя класса с нормализацией"""
        
#         # Нормализуем имя класса
#         class_name = self._normalize_identifier(entity_type)
        
#         # Делаем первую букву заглавной для соглашения об именах классов
#         if class_name:
#             class_name = class_name[0].upper() + class_name[1:]
        
#         return class_name

In [130]:
generator = SimpleLinkMLGenerator(
    entities_path="entities.parquet",
    relationships_path="relationships.parquet",
    max_entity_types=5,
    max_relations=5
)
linkml_schema = generator.generate_simple_schema()

Анализ типов сущностей...
Анализ отношений...
Построение схемы...


In [131]:
print(linkml_schema[:1000])

id: https://example.org/cad-graphrag-ontology
name: CADGraphRAGOntology
title: CAD GraphRAG Ontology
description: Автоматически сгенерированная онтология из GraphRAG данных CAD системы
license: https://creativecommons.org/publicdomain/zero/1.0/
imports:
- linkml:types
- core
prefixes:
  linkml: https://w3id.org/linkml/
  cad: https://example.org/cad/
  schema: http://schema.org/
default_prefix: cad
default_range: string
classes:
  CADExtractionResult:
    tree_root: true
    description: Result of extracting CAD entities and relationships from documentation
    attributes:
      parameter_list:
        description: semicolon-separated list of PARAMETER entities
        multivalued: true
        range: Parameter
      command_list:
        description: semicolon-separated list of COMMAND entities
        multivalued: true
        range: Command
      object_list:
        description: semicolon-separated list of OBJECT entities
        multivalued: true
        range: Object
      tool_l

In [132]:
with open("cad_graphrag_schema.yaml", "w", encoding="utf-8") as f:
    f.write(linkml_schema)

In [3]:
class SimpleLinkMLGeneratorMedium:
    """
    Класс построения LinkML схемы для OntoGPT на основе типов сущностей и отношений
    """
    
    def __init__(self, entities_path: str, relationships_path: str):
        self.entities_df = pd.read_parquet(entities_path)
        self.relationships_df = pd.read_parquet(relationships_path)
        
    def generate_simple_schema(self) -> str:
        """Вызов методов построения"""
        
        print("Анализ типов сущностей...")
        entity_types = self._analyze_entity_types()
        
        print("Анализ отношений...")
        relationship_types = self._analyze_relationship_types()
        
        print("Построение схемы...")
        schema = self._build_linkml_schema(entity_types, relationship_types)
        
        return yaml.dump(schema, sort_keys=False, allow_unicode=True, indent=2)
    
    def _analyze_entity_types(self) -> Dict:
        """Анализ типов сущностей и их атрибутов"""
        
        entity_types = {}
        
        # Группируем сущности по типу
        for entity_type in self.entities_df['type'].unique():
            if pd.isna(entity_type) or entity_type == '':
                continue
                
            type_entities = self.entities_df[self.entities_df['type'] == entity_type]
            
            # Подсчет статистики для примеров
            entity_types[entity_type] = {
                "description": f"Сущности типа {entity_type} из CAD системы",
                "count": len(type_entities),
                "sample_titles": type_entities['title'].head(3).tolist()
            }
        
        return entity_types
    
    def _analyze_relationship_types(self) -> Dict[str, Dict]:
        """Анализ типов отношений между сущностями"""
        
        relationship_types = {}
        
        # Анализируем описания отношений для группировки
        relationship_descriptions = self.relationships_df['description'].value_counts().head(20)
        
        for desc, count in relationship_descriptions.items():
            if pd.isna(desc) or desc == '':
                continue
                
            # Создаем имя для типа отношения на основе описания
            rel_name = self._create_relationship_name(desc)
            
            # Находим примеры source и target типов для этого отношения
            example_rels = self.relationships_df[
                self.relationships_df['description'] == desc
            ].head(10)
            
            source_types = set()
            target_types = set()
            
            for _, rel in example_rels.iterrows():
                source_entity = self.entities_df[self.entities_df['id'] == rel['source']]
                target_entity = self.entities_df[self.entities_df['id'] == rel['target']]
                
                if not source_entity.empty:
                    source_types.add(source_entity.iloc[0]['type'])
                if not target_entity.empty:
                    target_types.add(target_entity.iloc[0]['type'])
            
            relationship_types[rel_name] = {
                "description": desc,
                "count": count,
                "source_types": list(source_types),
                "target_types": list(target_types)
            }
        
        return relationship_types

    def _create_relationship_name(self, description: str, max_words: int = 3, max_length: int = 30) -> str:
        """Создание короткого имени отношения из описания с транслитерацией"""
        
        if not description or pd.isna(description):
            return "related_to"
        
        # Удаляем скобки, двоеточия, лишние пробелы
        clean_desc = re.sub(r'[()\[\]{}<>:;,.!?]', ' ', str(description))
        clean_desc = re.sub(r'\s+', ' ', clean_desc).strip().lower()
        
        words = clean_desc.split()
        
        if len(words) == 0:
            return "related_to"
        
        stop_words = {
            'и', 'или', 'а', 'в', 'во', 'из', 'от', 'до', 'для', 'к', 'на', 'по', 'с', 'со',
            'у', 'над', 'под', 'перед', 'через', 'без', 'о', 'об', 'около', 'возле'
        }
        
        meaningful_words = [word for word in words 
                           if word not in stop_words and len(word) > 2 and not word.isdigit()]
        
        if not meaningful_words:
            meaningful_words = words[:max_words]
        
        selected_words = meaningful_words[:max_words]
        
        transliterated_words = []
        for word in selected_words:
            translit_word = self._transliterate(word.strip())
            translit_word = re.sub(r'_+', '_', translit_word).strip('_')
            if translit_word:
                transliterated_words.append(translit_word)
        
        if not transliterated_words:
            return "related_to"
        
        result_name = '_'.join(transliterated_words[:max_words])
        
        if len(result_name) > max_length:
            result_name = result_name[:max_length].rsplit('_', 1)[0]
        
        result_name = re.sub(r'_+', '_', result_name).strip('_')
        
        if result_name and result_name[0].isdigit():
            result_name = f"rel_{result_name}"
        
        if len(result_name) < 3 or re.match(r'^\d+$', result_name):
            return "related_to"
        
        return result_name.lower()
    
    def _build_linkml_schema(self, entity_types: Dict, relationship_types: Dict) -> Dict:
        """Построение полной LinkML схемы с корректными URI и структурой для OntoGPT"""
        
        schema = {
            "id": "https://example.org/cad-graphrag-ontology",
            "name": "CADGraphRAGOntology",
            "title": "CAD GraphRAG Ontology",
            "description": "Автоматически сгенерированная онтология из GraphRAG данных CAD системы",
            "license": "https://creativecommons.org/publicdomain/zero/1.0/",
            "imports": [
                "linkml:types",
                "core"
            ],
            "prefixes": {
                "linkml": "https://w3id.org/linkml/",
                "cad": "https://example.org/cad/",
                "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
                "owl": "http://www.w3.org/2002/07/owl#",
                "schema": "http://schema.org/"
            },
            "default_prefix": "cad",
            "default_range": "string",
            "classes": {},
            "slots": {}
        }
        
        # 1. Создаем КОРНЕВОЙ класс для экстракции (tree_root)
        all_entity_slots = {}
        for entity_type in entity_types.keys():
            class_name = self._type_to_class_name(entity_type)
            slot_name = f"{class_name.lower()}_list"
            all_entity_slots[slot_name] = {
                "description": f"Список извлеченных сущностей типа {entity_type}",
                "multivalued": True,
                "range": class_name
            }
        
        # Добавляем слоты для отношений
        for rel_name in relationship_types.keys():
            rel_class_name = self._to_class_name(rel_name)
            slot_name = f"{rel_name}_list"
            all_entity_slots[slot_name] = {
                "description": f"Список извлеченных отношений {rel_name}",
                "multivalued": True,
                "range": rel_class_name
            }
        
        schema["classes"]["CADExtractionResult"] = {
            "tree_root": True,
            "description": "Результат извлечения информации из документации CAD системы",
            "attributes": all_entity_slots
        }
        
        # 2. Добавляем классы для каждого типа сущности
        for entity_type, type_info in entity_types.items():
            class_name = self._type_to_class_name(entity_type)
            
            # Определяем атрибуты класса
            attributes = {
                "label": {
                    "description": "Название сущности",
                    "slot_uri": "rdfs:label"
                },
                "description": {
                    "description": "Описание сущности",
                    "slot_uri": "schema:description"
                }
            }
            
            schema["classes"][class_name] = {
                "is_a": "NamedEntity",
                "description": type_info["description"],
                "id_prefixes": ["CAD"],  # Префикс для grounding
                "attributes": attributes,
                "annotations": {
                    "annotators": "sqlite:obo:mesh"
                }
            }
        
        # 3. Добавляем классы для отношений
        for rel_name, rel_info in relationship_types.items():
            rel_class_name = self._to_class_name(rel_name)
            
            # Определяем основные типы источника и цели
            source_type = rel_info["source_types"][0] if rel_info["source_types"] else "NamedEntity"
            target_type = rel_info["target_types"][0] if rel_info["target_types"] else "NamedEntity"
            
            source_class = self._type_to_class_name(source_type) if source_type != "NamedEntity" else "NamedEntity"
            target_class = self._type_to_class_name(target_type) if target_type != "NamedEntity" else "NamedEntity"
            
            attributes = {
                "source": {
                    "description": f"Исходная сущность отношения",
                    "range": source_class,
                    "annotations": {
                        "prompt": "Первая сущность в отношении"
                    }
                },
                "target": {
                    "description": f"Целевая сущность отношения",
                    "range": target_class,
                    "annotations": {
                        "prompt": "Вторая сущность в отношении"
                    }
                },
                "relation_type": {
                    "description": rel_info["description"],
                    "range": "string"
                }
            }
            
            schema["classes"][rel_class_name] = {
                "is_a": "CompoundExpression",
                "description": f"Отношение: {rel_info['description']}",
                "attributes": attributes
            }
        
        # 4. Определяем общие слоты (если нужны)
        schema["slots"]["label"] = {
            "description": "Название сущности",
            "range": "string",
            "slot_uri": "rdfs:label"
        }
        
        schema["slots"]["description"] = {
            "description": "Описание сущности",
            "range": "string",
            "slot_uri": "schema:description"
        }
        
        return schema
    
    def _type_to_class_name(self, entity_type: str) -> str:
        """Преобразование типа сущности в имя класса с транслитерацией"""
        
        if not entity_type or pd.isna(entity_type):
            return "UnknownEntity"
        
        # Транслитерируем тип сущности
        transliterated_type = self._transliterate(str(entity_type))
        
        # Замена пробелов и специальных символов
        clean_type = re.sub(r'[^a-zA-Z0-9]', '_', transliterated_type)
        
        # Удаляем повторяющиеся подчеркивания
        clean_type = re.sub(r'_+', '_', clean_type).strip('_')
        
        # Если тип начинается с цифры, добавляем префикс
        if clean_type and clean_type[0].isdigit():
            clean_type = "Entity_" + clean_type
        
        # Если после обработки строка пустая, используем fallback
        if not clean_type:
            clean_type = "UnknownEntity"
        
        # Приводим к CamelCase
        words = clean_type.split('_')
        return ''.join(word.capitalize() for word in words if word)
    
    def _to_class_name(self, name: str) -> str:
        """Преобразование произвольного имени в CamelCase имя класса"""
        
        if not name or pd.isna(name):
            return "UnknownRelation"
        
        # Разбиваем по подчеркиваниям
        words = str(name).split('_')
        return ''.join(word.capitalize() for word in words if word) + "Relationship"

    def _transliterate(self, text: str) -> str:
        """Транслитерация кириллических символов в латиницу"""

        if not text or pd.isna(text):
            return ""
        
        text = str(text).strip()
        
        translit_map = {
            'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'e',
            'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
            'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
            'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch',
            'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e', 'ю': 'yu', 'я': 'ya',
            'А': 'A', 'Б': 'B', 'В': 'V', 'Г': 'G', 'Д': 'D', 'Е': 'E', 'Ё': 'E',
            'Ж': 'Zh', 'З': 'Z', 'И': 'I', 'Й': 'Y', 'К': 'K', 'Л': 'L', 'М': 'M',
            'Н': 'N', 'О': 'O', 'П': 'P', 'Р': 'R', 'С': 'S', 'Т': 'T', 'У': 'U',
            'Ф': 'F', 'Х': 'Kh', 'Ц': 'Ts', 'Ч': 'Ch', 'Ш': 'Sh', 'Щ': 'Shch',
            'Ъ': '', 'Ы': 'Y', 'Ь': '', 'Э': 'E', 'Ю': 'Yu', 'Я': 'Ya'
        }
        
        result_chars = []
        prev_was_space = False
        
        for char in text:
            if char in translit_map:
                translit_char = translit_map[char]
                if translit_char:
                    result_chars.append(translit_char)
                    prev_was_space = False
            elif char.isalnum():
                result_chars.append(char)
                prev_was_space = False
            elif char.isspace() or char in ['-', '_', '/']:
                if not prev_was_space:
                    result_chars.append('_')
                    prev_was_space = True
        
        result = ''.join(result_chars)
        result = re.sub(r'_+', '_', result).strip('_')
        
        words = result.split('_')
        valid_words = [word for word in words if re.search(r'[a-zA-Z]', word)]
        
        if not valid_words:
            return result if result else "unknown"
        
        return '_'.join(valid_words).lower()


In [4]:
generator = SimpleLinkMLGeneratorMedium(
    entities_path="entities.parquet",
    relationships_path="relationships.parquet"
)
linkml_schema = generator.generate_simple_schema()

Анализ типов сущностей...
Анализ отношений...
Построение схемы...


In [5]:
print(linkml_schema[:1000])

id: https://example.org/cad-graphrag-ontology
name: CADGraphRAGOntology
title: CAD GraphRAG Ontology
description: Автоматически сгенерированная онтология из GraphRAG данных CAD системы
license: https://creativecommons.org/publicdomain/zero/1.0/
imports:
- linkml:types
- core
prefixes:
  linkml: https://w3id.org/linkml/
  cad: https://example.org/cad/
  rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
  rdfs: http://www.w3.org/2000/01/rdf-schema#
  owl: http://www.w3.org/2002/07/owl#
  schema: http://schema.org/
default_prefix: cad
default_range: string
classes:
  CADExtractionResult:
    tree_root: true
    description: Результат извлечения информации из документации CAD системы
    attributes:
      command_list:
        description: Список извлеченных сущностей типа COMMAND
        multivalued: true
        range: Command
      entity3dscene_list:
        description: Список извлеченных сущностей типа 3D SCENE
        multivalued: true
        range: Entity3dScene
      tool_list:
  

In [6]:
with open("cad_graphrag_schema_medium.yaml", "w", encoding="utf-8") as f:
    f.write(linkml_schema)

## OntoGPT извлечение данных

In [63]:
# pip install --upgrade jupyter ipywidgets tqdm setuptools

In [150]:
from ontogpt.engines.spires_engine import SPIRESEngine
from linkml_runtime.utils.schemaview import SchemaView
from ontogpt.io.template_loader import get_template_details, get_template_path
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [45]:
from ontogpt.engines.spires_engine import SPIRESEngine
import inspect

init_signature = inspect.signature(SPIRESEngine.__init__)
print(init_signature)

(self, template_details: tuple = None, template_class: linkml_runtime.linkml_model.meta.ClassDefinition = None, template_module: Optional[module] = None, schemaview: linkml_runtime.utils.schemaview.SchemaView = None, api_key: str = '', api_base: str = None, api_version: str = None, model: str = None, model_provider: str = None, annotators: Optional[Dict[str, List[oaklib.interfaces.text_annotator_interface.TextAnnotatorInterface]]] = None, mappers: Optional[List[oaklib.interfaces.basic_ontology_interface.BasicOntologyInterface]] = None, map_cache: Dict[str, List[str]] = <factory>, labelers: Optional[List[oaklib.interfaces.basic_ontology_interface.BasicOntologyInterface]] = None, client: ontogpt.clients.llm_client.LLMClient = None, dictionary: Dict[str, str] = <factory>, value_set_expansions: Dict[str, List[str]] = <factory>, named_entities: List[ontogpt.templates.core.NamedEntity] = <factory>, extracted_named_entities: List[ontogpt.templates.core.NamedEntity] = <factory>, auto_prefix: s

In [151]:
client = OpenAI(
    base_url=os.getenv('OPENAI_BASE_URL'),
    api_key=os.getenv('OPENAI_API_KEY'),
)

engine = SPIRESEngine(
    template_details=get_template_details(template="figure"), # SchemaView("cad_graphrag_schema.yaml"),
    model=os.getenv('MODEL_NAME'),
    # api_base=os.getenv('OPENAI_BASE_URL'),
    # api_key=os.getenv('OPENAI_API_KEY'),
    temperature=0.1,
    client=client
)

In [159]:
class OntoGPTExtractor:
    """
    Класс для извлечения онтологии
    """
    
    def __init__(self, schema_path: str, model: str = "openai/gpt-4.1", transliterate: bool = True):
        self.schema_path = schema_path
        self.model = model or os.getenv('MODEL_NAME', 'openai/gpt-4.1')
        self.base_url = os.getenv('OPENAI_BASE_URL', 'https://api.vsegpt.ru/v1')
        self.api_key = os.getenv('OPENAI_API_KEY')
        self.transliterate = transliterate

        if not self.api_key:
            raise ValueError("API ключ не найден. Убедитесь, что переменная OPENAI_API_KEY установлена в .env файле")

        self.client = OpenAI(
            base_url=os.getenv('OPENAI_BASE_URL'),
            api_key=os.getenv('OPENAI_API_KEY'),
        )
        
        self.engine = SPIRESEngine(
            schemaview=get_template_details(template="figure.FigureCaption"), #SchemaView(schema_path),
            model=os.getenv('MODEL_NAME'),
            temperature=0.1,
            client=self.client
        )
    
    def extract_from_documents(self, documents_directory: str) -> List[Dict]:
        """Извлечение онтологии из документов"""
        
        documents = self._load_documents(documents_directory)
        results = []
        
        for doc in documents:
            print(f"Обработка: {doc['filename']}")
            
            try:
                # Извлечение структурированных данных
                extraction = self.engine.extract_from_text(doc["content"])
                
                results.append({
                    "document": doc["filename"],
                    "extraction": extraction.dict() if hasattr(extraction, 'dict') else extraction,
                    "success": True,
                    "timestamp": pd.Timestamp.now().isoformat()
                })
                
                print(f"Успешно извлечено из {doc['filename']}")
                
            except Exception as e:
                print(f"Ошибка в {doc['filename']}: {str(e)}")
                results.append({
                    "document": doc["filename"],
                    "error": str(e),
                    "success": False
                })
        
        return results
    
    def _load_documents(self, directory: str) -> List[Dict]:
        """Загрузка документов из директории"""
        
        documents = []
        
        for file_path in Path(directory).glob("*.txt"):
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    
                    # Ограничиваем размер для больших файлов
                    if len(content) > 20000:
                        content = content[:20000] + "... [truncated]"
                    
                    documents.append({
                        "filename": file_path.name,
                        "content": content
                    })
                    
            except Exception as e:
                print(f"Ошибка чтения {file_path}: {e}")
        
        print(f"Загружено {len(documents)} документов")
        return documents
    
    def save_results(self, results: List[Dict], output_path: str):
        """Сохранение результатов извлечения"""
        
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        # Статистика
        successful = sum(1 for r in results if r.get('success', False))
        print(f"Результаты сохранены в {output_path}")
        print(f"Успешных извлечений: {successful}/{len(results)}")

In [160]:
extractor = OntoGPTExtractor("figure")

In [161]:
# extractor._load_documents("./graphragcad/input")

In [163]:
extraction_results = extractor.extract_from_documents("./cad_data2/input") #"./graphragcad/input")

Загружено 40 документов
Обработка: 1 Назначение программного модуля CAD.txt
Ошибка в 1 Назначение программного модуля CAD.txt: 'tuple' object has no attribute 'class_induced_slots'
Обработка: 2 Условия выполнения программного модуля CAD.txt
Ошибка в 2 Условия выполнения программного модуля CAD.txt: 'tuple' object has no attribute 'class_induced_slots'
Обработка: 3.1 Запуск программного модуля CAD.txt
Ошибка в 3.1 Запуск программного модуля CAD.txt: 'tuple' object has no attribute 'class_induced_slots'
Обработка: 3.2 Управление приложениями.txt
Ошибка в 3.2 Управление приложениями.txt: 'tuple' object has no attribute 'class_induced_slots'
Обработка: 3.3 Работа с документами.txt
Ошибка в 3.3 Работа с документами.txt: 'tuple' object has no attribute 'class_induced_slots'
Обработка: 3.3.1 Создание нового документа.txt
Ошибка в 3.3.1 Создание нового документа.txt: 'tuple' object has no attribute 'class_induced_slots'
Обработка: 3.3.2 Открытие документов.txt
Ошибка в 3.3.2 Открытие документо

In [164]:
extractor.save_results(extraction_results, "ontogpt_extraction_results.json")

Результаты сохранены в ontogpt_extraction_results.json
Успешных извлечений: 0/40


## Визуализация